# Exploring Hacker News Posts
Hacker News is a website for user-submitted stories primarily about computer science and entrepreneurship. A post made by a user is voted on and commented upon by other users.

Posts that have titles starting with "Ask HN" poses a question to the community. Posts that have titles starting with "Show HN" shows a project or something generally interesting to the community.

These two categories of posts will be targeted by this project. Attempts will be made to answer the following questions:
- Do "Ask NH" or "Show NH" receive more comments on average?
- Do posts created at a certain time receive more comments on average?

The following data set will be used: https://www.kaggle.com/hacker-news/hacker-news-posts
However, the data set used is actually a modified version of the forementioned, as provided in the Dataquest course. Posts without comments have been excluded and the size further reduced by randomly removing posts. The data set ends up being 20,000 rows in comarison to the original nearly 300,000.

## Accessing the data set
Let's start by accessing the file containing the data set and convert it to a list of lists, excluding the headers that is stored in a separate variable. Let's also print the headers and the first 5 rows of the data set:

In [1]:
from csv import reader
hn = list(reader(open('hacker_news.csv')))
headers = hn[0]
hn = hn[1:]

print(headers)
print('\n')
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


## Extracting "Ask HN" and "Show HN" posts
We can no begin exploring the data. As stated, we're only interested in posts that have titles starting with "Ask HN" or "Show HN" so we'll separate the data into three lists: `ask_posts`, `show_posts` and `other_posts`. To filter the posts in this way we'll use the built-in string method; `startswith`: https://docs.python.org/3/library/stdtypes.html#str.startswith

In [2]:
ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1].lower()
    if title.startswith('ask hn'):
        ask_posts.append(post)
    elif title.startswith('show hn'):
        show_posts.append(post)
    else:
        other_posts.append(post)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


As is apparent, most posts are neither an "Ask HN" or "Show HN" post. We have 1744 ask posts and 1162 show posts in total, and these are what we'll explore further.

## Average number of comments
Let's compute the average number of comments on an ask post and a show post respectively:

In [3]:
def avg_comments_in_posts(posts):
    n_comments = 0
    for post in posts:
        n_comments += int(post[4])
    return n_comments/len(posts)

avg_ask_comments = avg_comments_in_posts(ask_posts)
avg_show_comments = avg_comments_in_posts(show_posts)

print('Average ask-post comments:', avg_ask_comments)
print('Average show-post comments:', avg_show_comments)

Average ask-post comments: 14.038417431192661
Average show-post comments: 10.31669535283993


Ask-posts get on average 14.04 comments and show-posts 10.32. That is, on average ask-posts generate more comments and appear to be more engaging. We'll focus our remaining analysis on the ask-posts.

## Number of comments and ask-posts per hour
We will now explore what role posting time plays in the number of comments generated. Time will in this case be handled using the unit of hours. To aid us in this analysis we'll use the built-in datetime module: https://docs.python.org/3/library/datetime.html
Let's start by calculating the number of posts per hour as well as the total number of comments per hour. These will be stored in dictionaries; `counts_by_hour` and `comments_per_hour`.

In [4]:
import datetime as dt

result_list = []
for post in ask_posts:
    result_list.append([post[6], int(post[4])])

counts_by_hour = {}
comments_by_hour = {}
for post in result_list:
    time = dt.datetime.strptime(post[0], "%m/%d/%Y %H:%M").hour
    if time in counts_by_hour:
        counts_by_hour[time] += 1
        comments_by_hour[time] += post[1]
    else:
        counts_by_hour[time] = 1
        comments_by_hour[time] = post[1]

comments_by_hour

{9: 251,
 13: 1253,
 10: 793,
 14: 1416,
 16: 1814,
 23: 543,
 12: 687,
 17: 1146,
 15: 4477,
 21: 1745,
 20: 1722,
 2: 1381,
 18: 1439,
 3: 421,
 5: 464,
 19: 1188,
 1: 683,
 22: 479,
 8: 492,
 4: 337,
 0: 447,
 6: 397,
 7: 267,
 11: 641}

## Average number comments on ask-posts per hour
We will continue by calculating average comments on ask-posts by hour, using the forementioned dictionaries that are not populated. These will be stored in a list of list, `avg_by_hour`, in which the first value of each row represents the hour and the second value the corresponding average comments for that hour. The large size of the data set and the nature of the data makes it very unlikely that an hour is completely void of comments. Therefore, we won't bother handling this special case.

In [5]:
avg_by_hour = []
for i in range(0,24):
    avg_by_hour.append([i, comments_by_hour[i]/counts_by_hour[i]])

avg_by_hour

[[0, 8.127272727272727],
 [1, 11.383333333333333],
 [2, 23.810344827586206],
 [3, 7.796296296296297],
 [4, 7.170212765957447],
 [5, 10.08695652173913],
 [6, 9.022727272727273],
 [7, 7.852941176470588],
 [8, 10.25],
 [9, 5.5777777777777775],
 [10, 13.440677966101696],
 [11, 11.051724137931034],
 [12, 9.41095890410959],
 [13, 14.741176470588234],
 [14, 13.233644859813085],
 [15, 38.5948275862069],
 [16, 16.796296296296298],
 [17, 11.46],
 [18, 13.20183486238532],
 [19, 10.8],
 [20, 21.525],
 [21, 16.009174311926607],
 [22, 6.746478873239437],
 [23, 7.985294117647059]]

## Sorting and printing the values
With that done, let's try and make some sense of the numbers. First, we'll sort the list with regards to the average number of comments, descendingly. Then we'll print the top 5 hours in the sorted list.

To aid this, we'll first make a copy of the previous list, but where average comments now making out the first column and hour the second. This is done to enable us to easily use built-in algorithms on the average comments.

In [6]:
swap_avg_by_hour = []
for i in avg_by_hour:
    swap_avg_by_hour.append([i[1], i[0]])

swap_avg_by_hour

[[8.127272727272727, 0],
 [11.383333333333333, 1],
 [23.810344827586206, 2],
 [7.796296296296297, 3],
 [7.170212765957447, 4],
 [10.08695652173913, 5],
 [9.022727272727273, 6],
 [7.852941176470588, 7],
 [10.25, 8],
 [5.5777777777777775, 9],
 [13.440677966101696, 10],
 [11.051724137931034, 11],
 [9.41095890410959, 12],
 [14.741176470588234, 13],
 [13.233644859813085, 14],
 [38.5948275862069, 15],
 [16.796296296296298, 16],
 [11.46, 17],
 [13.20183486238532, 18],
 [10.8, 19],
 [21.525, 20],
 [16.009174311926607, 21],
 [6.746478873239437, 22],
 [7.985294117647059, 23]]

In [7]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print("Top 5 hours regarding average number of comments among 'Ask HN'-posts:\n")
for i in range(0,5):
    print("{}: {:.2f} average comments per post".format
          (dt.datetime.strptime(str(sorted_swap[i][1]), '%H').strftime('%H:%M'), sorted_swap[i][0]))

Top 5 hours regarding average number of comments among 'Ask HN'-posts:

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that undoubtedly has the highest average comments per post is 15, coming in at 38.59.

According to the documentation for the data set, the time zone utilized is eastern time in the US (GMT -4). In Sweden, the time zone is central european time (CET). CET is 6 hours ahead of the time zone used in this data set. That is, the corresponding time in Sweden for the hour with the highest number of average comments on "Ask HN"-posts is 21.

## Conclusion
In this project, "Ask HN" and "Show HN" posts were analyzed to determine which on average gets more comments. Our analysis showed that "Ask HN" came out ahead with an average of 14.04 average comments compared to "Show HN" that showed an 10.32 average. This could indicate that such posts tend to generate a greater general engagement in the community. However, it's worth noting that it could also be a result of "Ask HN" explicitly requesting answers (comments) whereas "Show HN" does not.

We then continued on focusing on the "Ask HN" posts to try and determine if the posting time had any effect on the number of comments received. As it turns out, posting time seem to have a large impact on the number of comments made on average. The time that on average generates the highest amount comments on an "Ask HN" post is undoubtedly between 15:00 and 16:00 (between 21:00 and 22:00 in Sweden).

However, keep in mind that posts completely lacking comments were excluded from this data set. Therefore, it is more accurate modifying the previous statement to saying that it regards specifically "Ask HN" posts that has actually has gotten comments.